# aggregate results

In [1]:
import numpy as np
import pandas as pd
import xgboost
from sklearn.metrics import roc_auc_score

/opt/conda/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
dl = [pd.read_csv('DS_2_train.csv', index_col=0)]
dty = "/data/episode2/"
# dl.append(pd.read_hdf("data-r1.h5", "train"))
dl.append(pd.read_hdf(dty + "data-r1.h5", "train"))
for i in range(2,7):
    dl.append(pd.read_hdf(dty + "data-r{}.h5".format(i), "train"))
for i in range(6):
    dl.append(pd.read_hdf("hiss-r{}.h5".format(i), "train"))
d0 = pd.concat(dl, axis=1)
print(d0.shape)

/opt/conda/anaconda3/lib/python3.6/site-packages/numpy/lib/arraysetops.py:463: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


(9811609, 27)


In [3]:
el = [pd.read_csv('DS_2_test.csv', index_col=0)]
el.append(pd.read_hdf(dty + "data-r1.h5", "test"))
for i in range(2,7):
    el.append(pd.read_hdf(dty + "data-r{}.h5".format(i), "test"))
for i in range(6):
    el.append(pd.read_hdf("hiss-r{}.h5".format(i), "test"))
e0 = pd.concat(el, axis=1)
print(e0.shape)

/opt/conda/anaconda3/lib/python3.6/site-packages/numpy/lib/arraysetops.py:463: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


(9789204, 25)


In [4]:
e0.columns

Index(['X', 'Y', 'Z', 'TX', 'TY', 'chi2', 'brick_number', 'dfore', 'dback',
       'dfore2', 'dback2', 'dfore3', 'dback3', 'dfore4', 'dback4', 'dfore5',
       'dback5', 'dfore6', 'dback6', 'cnt0', 'cnt1', 'cnt2', 'cnt3', 'cnt4',
       'cnt5'],
      dtype='object')

In [5]:
d0.columns

Index(['event_id', 'X', 'Y', 'Z', 'TX', 'TY', 'chi2', 'signal', 'brick_number',
       'dfore', 'dback', 'dfore2', 'dback2', 'dfore3', 'dback3', 'dfore4',
       'dback4', 'dfore5', 'dback5', 'dfore6', 'dback6', 'cnt0', 'cnt1',
       'cnt2', 'cnt3', 'cnt4', 'cnt5'],
      dtype='object')

In [6]:
e0.head()

,X,Y,Z,TX,TY,chi2,brick_number,dfore,dback,dfore2,...,dfore5,dback5,dfore6,dback6,cnt0,cnt1,cnt2,cnt3,cnt4,cnt5
index,,,,,,,,,,,,,,,,,,,,,
0,57690.351562,40314.937500,47841.0,-0.268973,0.039472,1.113268,9,2.209555e+06,7.706569e+05,4.016672e+06,...,1.530483e+06,4.614029e+06,4.185280e+06,1.798303e+06,1.0,96.0,11825.0,1409671.0,169377264.0,2.061479e+10
1,37949.566406,72453.781250,50427.0,0.092281,-0.026488,2.673564,87,2.029628e+04,1.908238e+06,1.167201e+06,...,2.468699e+06,3.873801e+06,4.700463e+06,4.015226e+06,2.0,125.0,12553.0,1407005.0,164057376.0,1.948317e+10
2,42897.289062,16920.482422,19395.0,-0.069568,0.044336,2.672163,38,2.757374e+06,1.298424e+06,2.837138e+06,...,3.510851e+06,3.370621e+06,6.322015e+05,1.791147e+06,2.0,105.0,9587.0,1100783.0,135250814.0,1.691486e+10
3,54251.625000,48776.027344,20688.0,0.175112,-0.184691,2.369353,39,1.548522e+06,7.524258e+05,1.144364e+06,...,1.130592e+06,6.724501e+06,2.372434e+06,5.350869e+06,2.0,128.0,13301.0,1499538.0,175073608.0,2.088176e+10
4,62038.414062,26468.835938,25860.0,-0.259970,-0.155785,2.490212,36,1.190425e+05,1.646965e+06,6.390038e+05,...,7.617188e+06,6.051434e+06,2.451678e+06,2.748829e+06,1.0,94.0,9908.0,1153833.0,140408634.0,1.734247e+10


In [7]:
d0.head()

,event_id,X,Y,Z,TX,TY,chi2,signal,brick_number,dfore,...,dfore5,dback5,dfore6,dback6,cnt0,cnt1,cnt2,cnt3,cnt4,cnt5
index,,,,,,,,,,,,,,,,,,,,,
0,35130,50550.007812,66924.796875,65943.0,0.639179,-0.086816,0.583319,1.0,53,1.585658e+05,...,1.194471e+07,1.780062e+05,1.000000e+06,1.286357e+06,8.0,452.0,38502.0,3863845.0,416073347.0,4.649980e+10
1,-999,48811.390625,65584.648438,12930.0,0.569996,0.294196,1.582101,0.0,49,1.121876e+06,...,9.551373e+06,4.645293e+06,1.411788e+07,5.929448e+06,1.0,72.0,7696.0,930396.0,117267251.0,1.495537e+10
2,-999,30120.525391,28329.425781,21981.0,-0.366648,-0.379520,1.292502,0.0,39,1.072219e+05,...,1.702247e+07,1.874143e+06,2.716491e+07,6.713113e+06,1.0,56.0,7168.0,925015.0,117055868.0,1.469201e+10
3,-999,67186.773438,23033.931641,68529.0,-0.434561,0.502130,2.832125,0.0,65,1.063053e+06,...,1.000000e+06,7.125893e+06,1.000000e+06,1.057944e+06,1.0,65.0,7767.0,982534.0,125283734.0,1.592927e+10
4,-999,64832.343750,64279.328125,7758.0,0.139993,-0.417299,2.959314,0.0,13,1.009342e+06,...,2.714149e+06,1.649428e+06,8.194248e+06,1.950152e+06,1.0,67.0,7935.0,991420.0,125263663.0,1.587399e+10


In [8]:
# e0[ofeature].to_hdf("shared.h5", "test", complevel=9)

In [9]:
features = ["X", "Y", "Z", "TX", "TY", "chi2"] + ["dfore{}".format(i) for i in range(2,7)]
features += ["dback{}".format(i) for i in range(2,7)]
features += ["dback", "dfore"]
features += ["cnt{}".format(i) for i in range(4)]
features2 =  ["X", "Y", "Z", "TX", "TY", "chi2"]
tb = [0, 1, 2, 9, 14, 32, 40, 55, 75, 79, 80, 81, 82, 85, 93]
test = d0.loc[d0.brick_number.isin(tb)]
train = d0.loc[~d0.brick_number.isin(tb)]

In [10]:
print(len(train.query("signal==1")), len(train.query("signal==0")))
train_s = train.query("signal==1")
train1 = pd.concat((train_s, train.query("signal==0").sample(len(train_s))))
print(len(train1))

138529 8298484
277058


In [17]:
d0_s = d0.query("signal==1")
d01 = pd.concat((d0_s, train.query("signal==0").sample(len(d0_s))))
print(len(d0_s), len(d01))

162209 324418


In [22]:
xgb1 = xgboost.XGBClassifier(n_estimators=500, nthread=20, max_depth=4, reg_alpha=0, reg_lambda=0.5)
xgb1.fit(train[features], train.signal)

XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
       gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=4,
       min_child_weight=1, missing=None, n_estimators=500, nthread=20,
       objective='binary:logistic', reg_alpha=0, reg_lambda=0.5,
       scale_pos_weight=1, seed=0, silent=True, subsample=1)

In [23]:
pred0 = xgb1.predict_proba(test[features])[:,1]
print(roc_auc_score(test.signal, pred0))

0.997267414498


In [24]:
# xgb2 = xgboost.XGBClassifier(n_estimators=500, nthread=20, max_depth=4, reg_lambda=0.5)
# xgb2.fit(d0[features], d0.signal)

In [25]:
pred = xgb1.predict_proba(e0[features])[:,1]

In [26]:
def tresult(prediction, rev):
    baseline = pd.DataFrame(prediction, columns=['Prediction'])
    baseline.index.name = 'Id'
    baseline.to_csv('{}.csv.gz'.format(rev), header=True, compression="gzip")
tresult(pred, "noratio")